# Réception, décodage et affichage d’une image satellite METEOR-M2
                               PARTIE 2: Décompression JPEG et affichage de l'image

Dans la partie précédente nous avons réceptionné le signal provenant du satellite météorologique
Meteor-M2, puis nous avons traité ce signal afin de pouvoir récupérer les données images compressées
au format JPEG. L’objectif de cette partie est de décoder plusieurs imagettes de 8x8 pixels et de
vérifier que le décodage correspond bien à une partie de l’image complète décodée par le programme
"meteor-decoder".


# I. EXTRACTION DES DONNÉES UTILES

La fonction extract(fichier) contenue dans "extract_viterbi.py" effectue les actions suivantes :

— Ouvre le fichier binaire issue du décodage précédent (viterbi).

— Réalise la conversion en hexadécimal comme précédemment.

— Extrait les octets compris entre deux mots de synchronisation (1acffc1d), réalise la conversion en
décimal et place chaque ligne dans un array en forme de tableau.

— Pour chaque ligne, il affiche l’endroit à partir duquel commencent les informations à exploiter.

Toutes ces étapes sont visibles dans le fichier "extract_viterbi.py" et ne présentent pas de difficultés
particulières. Elles ont été réalisées en suivant les explications de [2] p6-8.

1. **Effectuer les opérations ci-dessus en important le fichier :**


In [1]:
#import extract_viterbi


Si vous souhaitez changer le fichier à traiter, ouvrir le fichier extract_viterbi.py et modifier l’instruction (autour de la ligne 100) :

extract_viterbi.extraction ("viterbi2.bin")


In [2]:
#import importlib
#importlib.reload(extract_viterbi) # A refaire apres chaque modification du fichier


![title](images/viterbi2.png)

Quasiment toutes les lignes commencent par la valeur 64, 65 ou 68. Ce sont les APID (Packet
identifier) de l’instrument à bord qui transmet les données (On rappelle qu’il y a bien trois gammes de
longueurs d’onde). De temps en temps, une ligne commence par la valeur 70. Il s’agit d’une information
de télémétrie.

2. **Identifier une ligne de télémétrie et trouver l’heure envoyée par le satellite à partir de
l’octet 21 (en comptant l’octet 70 comme 0). Est-ce cohérent avec l’information donnée
par meteor-decoder : "Onboard time : 12 :07 :25.928". Notre décodage est-il validé jusque-là ?**


*Répondre ici*

Nous allons nous intéresser dans la suite à la ligne 26, concernant des données provenant de
l’instrument ayant pour APID 65. Sur une telle ligne, l’octet 18 correspond au facteur de qualité
Q (qui nous servira plus tard) et les octets suivants concernent les données de l’image envoyée.

3. **Identifier la ligne en question, donner la valeur du facteur de qualité Q pour cette ligne
ainsi que les huit premiers octets correspondants aux données de l’image.**


*Répondre ici*

# II. DÉCODAGE DE HUFFMAN
Commençons par revoir et approfondir un peu le principe de la compression JPEG :

![title](images/jpeg.png)
![title](images/jpeg2.png)
![title](images/jpeg3.png)
![title](images/jpeg4.png)
4. **Décoder les 7 premiers octets des données de la ligne 26 sachant qu’ils sont encodés selon
l’algorithme de Huffman (commencer par les convertir en binaire sur 8 bits).**


*Répondre ici*

5. **Vérifier que votre décodage est correct en utilisant la fonction ci-dessous. Pourquoi a
t-on ajouté 1010 à la fin ?**


In [5]:
 extract_viterbi.huffman("1111100001011011010110001111100000000111111101100001" + "1010")


NameError: name 'extract_viterbi' is not defined

*Répondre ici*

6. **Réaliser le décodage de la ligne 26 complète sachant qu’elle contient un MCU complet
de 14 imagettes en utilisant la fonction :**


In [4]:
extract_viterbi.huff(line,nbr)


NameError: name 'extract_viterbi' is not defined

où line correspond à la ligne souhaitée et nbr au nombre d’imagettes à décoder.
Lors du décodage manuel, on se rend compte que certains octets sont codés sur plus de huit bits,
étrange pour une méthode de compression ? Pour l’exemple traité à la main plus haut, le gain en
compression n’est pas flagrant... Qu’en est-il lorsqu’on considère un plus grand nombre d’octets ?
7. **La fonction précédente affiche dans la console le nombre d’octets compressés et le nombre
d’octets décompressés. Calculer le pourcentage de compression gagné avec la méthode de
Huffman. Commenter.**


*Répondre ici*

8. **Calculer le taux de compression associé à cette zone (on ne tient pas compte des tables
à joindre au fichier ni du facteur de qualité à transmettre également). Commenter.**


*Répondre ici*

# III. DES ZIGS ET DES ZAGS
9. **Compléter les 15 premières valeurs de la première matrice de la ligne 26 en utilisant la
lecture en zigzag :**


-233 ... ... ... ... 

...  ... ... ... ...

... ... ... ... ...

... ... ...

... ...

...

Puis vérifier avec la fonction :


In [ ]:
extract_viterbi.zigzag(ligne,nbr_total,matrice)


ligne : le numéro de la ligne,
nbr_total : le nombre d’imagettes dans la ligne considérée,
matrice : le numéro de la matrice à afficher

10. **Faire afficher la matrice suivante. En vérifiant avec la liste affichée plus haut dans la
console, que constate-t-on pour la valeur DC ? Prédire la valeur DC de la troisième
matrice.**


*Répondre ici*

# IV. QUANTIFICATION INVERSE
Commencer par créér une variable DCT qui va stocker la valeur de la première matrice en zigzag :


In [3]:
DCT = extract_viterbi.zigzag(26,14,0)


NameError: name 'extract_viterbi' is not defined

Pour la quantification on utilise une matrice classique de quantification que l’on nomme HTK (voir
ci-dessous)
Cette matrice est modifiée en fonction du facteur de qualité Q selon les instructions suivantes :

— Si Q est compris entre 20 et 50 : F = 5000 / Q
 et F = 200 − 2 × Q sinon.

— On en déduit une nouvelle matrice PTK dont les éléments sont ceux de HTK multiplié par F/100.

— La matrice originale avant quantification F0 est obtenue en multipliant élément par élément DCT
et PTK.

11. **Ouvrir le fichier extract_viterbi.py et compléter la fonction quantif(DCT,Q) (vers les
lignes 520) en tenant compte des instructions ci-dessus (cf Figure 7).**

![title](images/fig7.png)

12. **Executer la fonction pour afficher la matrice F0 et la stocker dans une variable F0 :**

En ayant bien pris soin de remplacer la valeur de Q par celle trouvée à la question 18.



In [ ]:
import importlib # Faire qu’une fois
importlib.reload(extract_viterbi)   # Refaire apres chaque modification
F0 = extract_viterbi.quantif(DCT,Q)


# V. TRANSFORMÉE EN COSINUS DISCRÈTE INVERSE
La formule permettant de calculer la DCT inverse est donnée :

![title](images/dct.png)


13. **Toujours dans le même fichier (cf Figure 8), compléter la fonction suivante tcdi(F0) à
l’aide de la formule ci-dessus (π = math.pi) :**


![title](images/fig8.png)

14. **Executer la fonction pour afficher la matrice S et la stocker dans une variable :**


In [ ]:
import importlib # Faire qu’une fois
importlib.reload(extract_viterbi)   # Refaire apres chaque modification
S = extract_viterbi.tcdi(F0)


15. **Pourquoi a t-on des nombres négatifs ? Que reste t-il à faire ? Le réaliser.**


# VI. ENFIN DES IMAGES !
On utilise le module image de PIL :


In [ ]:
import PIL
image = PIL.Image.fromarray(S)
image.show()

16. **Qu’observez-vous ?**


*Répondre ici*

On va essayer de faire mieux en affichant les 14 imagettes de la ligne 25 :

17. **En récapitulant toutes les étapes précédentes dans une boucle, remplir une liste s=[]
contenant les 14 matrices S. Puis à l’aide de la fonction np.concatenate((a,b), axis=1)
rassembler toutes les matrices en une seule ligne et afficher l’image.**


In [ ]:
s = []
for i in range(14):
    DCT = ...
    F0 = ...
    S = ...
    S = ...
    s.append(S)

In [ ]:
import numpy as np
S=s[0]
for i in range(...):
    S = np.concatenate((S,....), axis=1)
image = PIL.Image.fromarray(S)
image.show()

18. **Reconnaissez vous cet endroit sur l’image entière ? Enregistrer l’image au format bmp
(clic - File - Save) et à l’aide de The Gimp par exemple, superposez cette imagette sur
l’image originale pour montrer la correspondance.**

![title](images/fin.png)


## Références :

[1] J.-M Friedt, Décodage d’images numériques issues de satellites météorologiques en orbite basse :
le protocole LRPT de Meteor-M2 (partie 1/2), janvier 2019, http ://jmfriedt.free.fr/glmf_meteor1.pdf

[2] J.-M Friedt, Décodage d’images numériques issues de satellites météorologiques en orbite basse :
le protocole LRPT de Meteor-M2 (partie 2/2), mars 2019, http ://jmfriedt.free.fr/glmf_meteor1.pdf

[3] Memon Nasir, ... Ansari Rashid, in Handbook of Image and Video Processing (Second Edition),
2005, The JPEG Lossless Image Compression Standards, https ://www.sciencedirect.com/topics/engineering/huffm
table

